![](https://raw.githubusercontent.com/JannisNe/timewise/refs/heads/main/timewise.png)
# Infrared light curves from WISE data

This package downloads WISE data for positions on the sky and stacks single-exposure photometry per visit. It is designed to do so for efficiently for large samples of millions of objects. For more info see the repo [here](https://github.com/JannisNe/timewise).

## Prerequisites
Python version 3.11, 3.12 or 3.13.

If you want to not only download individual exposure photometry but also stack detections per visit (see below),
you must have access to a running [MongoDB](https://www.mongodb.com/)* **. If you are running this notebook online you can make a free MongoDB Atlas account [here](https://www.mongodb.com/products/platform). Also make sure to allow access from your current IP address in the "Network Access" section of your MongoDB Atlas dashboard (or `0.0.0.0/0` to allow access from anywhere).

<sub>* On MacOS have alook at the custom `brew` tap
[here](https://github.com/mongodb/homebrew-brew)
to get the MongoDB community edition. </sub>

<sub>** On some systems this is not straight forward to set up. `timewise` requires it nevertheless as an integral part of the AMPEL system which is used to efficiently schedule and store the stacking of lightcurves. If you do not foresee a big overhead in calculating lightcurves for a sample of O(1000) objects, a more lightweight package might be more applicable. </sub>


## Installation

### If you use timewise only for downloading
The package can be installed via `pip` (but make sure to install the v1 pre-release):
```bash
pip install --pre timewise==1.0.0a10
```
### If you use timewise also for stacking individual exposures
You must install with the `ampel` extra:
```bash
pip install --pre 'timewise[ampel]==1.0.0a10'
```
To tell AMPEL which modules, aka units, to use, build the corresponding configuration file:
```bash
ampel config build -distributions ampel timewise -stop-on-errors 0 -out <path-to-ampel-config-file>
```

We will install with the `ampel` dependency:

In [ ]:
!pip install timewise[ampel]==1.0.0a10

## Command line interface

In [ ]:
!timewise --help

The input is a CSV file with at least three columns:  
- `orig_id`: an original identifier that **must** be an integer (for now)
- `ra`, `dec`: Right Ascension and Declination

As an example, let's use the first ten quasars from the Quaia catalog [(Kate Storey-Fisher et al 2024 ApJ 964 69)](https://iopscience.iop.org/article/10.3847/1538-4357/ad1328):

In [ ]:
!pip install wget

In [ ]:
from pathlib import Path

import pandas as pd
import wget
from astropy.table import Table

# download the quaia catalog from zenodo
working_directory = Path("./").resolve()
download_file = working_directory / "quaia_G20.0.fits"
quaia_url = "https://zenodo.org/records/10403370/files/quaia_G20.0.fits?download=1"
wget.download(quaia_url, str(download_file))

# open the catalog and save the first ten object to a CSV file
csv_file = download_file.with_suffix(".csv")
t = Table.read(download_file).to_pandas().iloc[:10]
t["orig_id"] = t["source_id"]
t.to_csv(csv_file, index=False)
download_file.unlink()

# let's display the content
pd.read_csv(csv_file)

`timewise` is configured with a YAML file. Below is a sensible default* which will use all single exposure photometry from AllWISE and NEOWISE in a 6 arcsecond radius around each source:

<sub>
* The MongoDB URI will depend on your specific set-up. If you installed a local instance with e.g. `brew` you do not have to specify this field to use the default mongodb://localhost:27017
</sub>

In [ ]:
yaml_string = """
download:
  input_csv: {path_to_input}
  n_per_chunk: 100000

  backend:
    type: filesystem
    base_path: {path_to_working_directory}

  queries:
    - type: positional
      radius_arcsec: 6
      table:
        name: allwise_p3as_mep
      columns:
        - ra
        - dec
        - mjd
        - cntr_mf
        - w1mpro_ep
        - w1sigmpro_ep
        - w2mpro_ep
        - w2sigmpro_ep
        - w1flux_ep
        - w1sigflux_ep
        - w2flux_ep
        - w2sigflux_ep

    - type: positional
      radius_arcsec: 6
      table:
        name: neowiser_p1bs_psd
      columns:
        - ra
        - dec
        - mjd
        - allwise_cntr
        - w1mpro
        - w1sigmpro
        - w2mpro
        - w2sigmpro
        - w1flux
        - w1sigflux
        - w2flux
        - w2sigflux

ampel:
  mongo_db_name: {mongodb_name}
  uri: {uri}
"""

Let's fill the variables in the yaml configuration and save it to a file. The `mongo_uri` will depend on your set-up. If you have a local MongoDB instance, it is likely running at `mongodb://localhost:27017`. If you are using MongoDB Atlas, you can find the connection string in the "Connect" section of your cluster dashboard.

In [ ]:
mongo_uri = ...
formatted_yaml_string = yaml_string.format(
    path_to_input=str(csv_file),
    path_to_working_directory=str(working_directory / "timewise_data"),
    uri=mongo_uri,
    mongodb_name="quaia_timewise_demo",
    )

timewise_config_file = working_directory / "quaia_timewise.yml"
with timewise_config_file.open("w") as f:
  f.write(formatted_yaml_string)

This configuration file will be the input to all subcommands. Downloading and stacking can be run together or separate.


### Query and download the data:


In [ ]:
!timewise download --help

In [ ]:
!timewise download quaia_timewise.yml

You can find the query results in the configure directory. The FITS files contain the raw photometry:

In [ ]:
!ls timewise_data/

In [ ]:
Table.read("./timewise_data/download_chunk0000_positional_neowiser_p1bs_psd_db292e4d5b65426a060a4ec349984298948e16e89f998b6bb4e965fd20a299a6.fits")

### Stack individual exposure by visits

As mentioned above, `timewise` is designed to compute lightcurves for many objects. It used the AMPEL system to do that. To tell AMPEL which modules ("units" in AMPEL terms) to use, build the corresponding configuration file:

In [ ]:
!ampel config build -distributions ampel timewise -stop-on-errors 0 -out ampel_config.yml

The ampel config is built assuming a local MongoDB instance at `mongodb://localhost:27017`. Let's use a remote MongoDB instead.

In [ ]:
with open("ampel_config.yml", "r") as f:
  config_string = f.read()

config_string_remote_db = config_string.replace("mongodb://localhost:27017", mongo_uri)

with open("ampel_config.yml", "w") as f:
  f.write(config_string_remote_db)

If you are using MongoDB Atlas: you have to delete the `storageEngine` specification in the config file!

Some `timewise` utility to help you set up your AMPEL job file:

In [ ]:
!timewise prepare-ampel quaia_timewise.yml

This imports the input into the MongoDB as well as create a standard AMPEL job file:

In [ ]:
!cat quaia_timewise_ampel_job.yml

This might be confuding at first! Here is an attempt of an explanation. AMPEL runs so-called units (`unit` keyword in the job file). Each unit can be configured individually (`config` keyword in the configuration file).

The workflow above defines two tasks:

#### 1. `t0`
This loads the downloaded data from the configured timewise data directory (`TimewiseFileLoader`) and supplies the data per object in the format demanded by AMPEL (`TimewiseAlertSupplier`). Each individual datapoint will be ingested into the database, making sure no duplicate data is present (`TiMongoMuxer`). This is especially helpful for duplicate entris in the AllWISE MEP database (see [this](https://irsa.ipac.caltech.edu/data/WISE/docs/release/AllWISE/expsup/sec1_3.html) for more info). The data per object will be selected as the closest cluster at the position of the parent sample object as explained in [Necker at al. (2024)](https://www.aanda.org/articles/aa/abs/2025/03/aa51340-24/aa51340-24.html) (`T1HDBSCAN`). For each comiled set of datapoints, The calculation of the stacked lightcurve is scheduled (`T2StackVisits`).


#### 2. `t2`
All scheduled calculations of tier 2 (T2) units is executed.

`T1HDBSCAN` uses the position of the parent sample objects stored in the database. We have to tell the config again which MongoDB we are using:

In [ ]:
import yaml

with open("quaia_timewise_ampel_job.yml", "r") as f:
  ampel_job = yaml.safe_load(f)

ampel_job["task"][0]["config"]["directives"][0]["ingest"]["mux"]["combine"][0]["config"]["mongo"] = mongo_uri
ampel_job["task"][0]["config"]["directives"][0]["ingest"]["mux"]["combine"][0]["config"]["plot"] = False

with open("quaia_timewise_ampel_job.yml", "w") as f:
  yaml.safe_dump(ampel_job, f)

We can now run the AMPEL job:

In [ ]:
!ampel job -schema quaia_timewise_ampel_job.yml -config ampel_config.yml -task 2

Some diagnostic plots:

In [ ]:
!mkdir ./timewise_plots
!timewise plot quaia_timewise.yml 10892037246720 54838142692736 ./timewise_plots

### Further process stacked lightcurves

#### Option 1: Export
If you want to extract the stacked lightcurves to further process them with your own framework, do this:

In [ ]:
!timewise export quaia_timewise.yml ./timewise_export
!ls ./timewise_export

#### Option 2: Leverage AMPEL power

Due to AMPEL's inherent modularity, you can add your own AMPEL units to process the lightcurves, extract features, aggregate results, etc. Below is an example job file from ongoing work in [this repo](https://github.com/JannisNe/airgn):

```yaml
channel:
- access:
  - ZTF_PUB
  name: wise
  policy: []
  version: 0
mongo:
  prefix: desi_agn_test_var_metrics
  reset: false
name: timewise
task:
- config:
    compiler_opts: TiCompilerOptions
    directives:
    - channel: wise
      ingest:
        mux:
          combine:
          - state_t2:
            - unit: T2StackVisits
            - unit: T2CalculateVarMetrics
              config:
                t2_dependency:
                  - unit: T2StackVisits
            unit: T1SimpleCombiner
          unit: TiMongoMuxer
    iter_max: 1000000
    shaper: TiDataPointShaper
    supplier:
      config:
        dpid: hash
        loader:
          config:
            timewise_config_file: $AIRGNSOURCE/airgn/desi/desi_agn_value_added_catalog.yml
            stock_id_column_name: orig_id
          unit: TimewiseFileLoader
      unit: TimewiseAlertSupplier
  multiplier: 1
  title: t0
  template:
    live:
      - resolve_run_time_aliases
      - hash_t2_config
  unit: AlertConsumer

- config:
    log_profile: default
  multiplier: 1
  title: t2
  unit: T2Worker

- title: PlotChi2
  unit: T3Processor
  config:
    raise_exc: true
    supply:
      unit: T3DefaultBufferSupplier
      config:
        select:
          unit: T3StockSelector
          config:
            channel: "wise"
        load:
          unit: T3SimpleDataLoader
          config:
            directives:
              - T2DOC
              - STOCK
            channel: "wise"
        chunk_size: 10000
    stage:
      unit: T3SimpleStager
      config:
        execute:
          - unit: VarMetricsVsAGN
            config:
              path: $AIRGNDATA/desi_value_added_catalog/plots/test_var_vs_agn
              input_mongo_db_name: desi_agn_vac
              file_format: "pdf"
              n_points_bins: [25, 30]
              metric_names:
                - pearsons_r
                - red_chi2
                - normalized_excess_variance
                - inverse_von_neumann_ratio
```

